In [ ]:
%matplotlib ipympl

In [ ]:
import sep
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("../imaging")
from convenience_functions import show_image

In [ ]:
from phot_utils import plot_sources, swap_byteorder, xmatch, to_mag, get_atm_extinction

In [ ]:
import astropy.units as u

In [ ]:
from astropy import coordinates

In [ ]:
from astropy.nddata import CCDData
from matplotlib.patches import Ellipse

# Data loading

In [ ]:
img_std = CCDData.read("../imaging/OB0004/reduced/stds/0003882223-20230709-OSIRIS-OsirisBroadBandImage-astrom.fits")

In [ ]:
plate_scales = img_std.wcs.proj_plane_pixel_scales()

In [ ]:
plate_scale = (plate_scales[0] + plate_scales[1]) / 2

plate_scale, plate_scale.to("arcsec") # should be close to 0.254 arcsec / pixel

plate_scale = 0.254

In [ ]:
gain = img_std.header["GAIN"]
airmass = img_std.header["airmass"]
filt = img_std.header["filter2"]

In [ ]:
exp_time_std = img_std.header["EXPTIME"]

In [ ]:
atm_ext, atm_ext_err = get_atm_extinction(airmass, filt)
atm_ext, atm_ext_err

In [ ]:
filt_letter = filt.split("Sloan_")[1]

In [ ]:
def get_ugriz(values):
    """From Sloan r apparent magnitude and a small list of the color,
    this function estimates the magnitude for each Sloan band.

    Args:
        values (list): List with Sloan r magnitude and colors. The order
        must be Sloan r, u-g , g-r, r-i, i-z. 

    Returns:
        float: Five magnitudes: Sloan u, Sloan g, Sloan r, Sloan i, Sloan z. 
    """
    r, ug, gr, ri, iz = values
    g = gr + r
    u = ug + g
    i = -ri + r
    z = -iz + i
    return u, g, r, i, z

In [ ]:
phot_standards = {
    "G158-100": ["00 33 54.60", "-12 07 58.9", 14.691, 1.101, 0.510, 0.222, 0.092],
    "SA92-282": ["00 56 46.86",	"+00 38 30.9", 12.936, 1.000, 0.136, 0.021, -0.009],
    "Feige22": ["02 30 16.62",	"+05 15 50.6", 13.024, 0.050, -0.333, -0.303, -0.273],
    "SA95-193": ["03 53 20.59", "+00 16 34.7", 13.844, 2.489, 1.097, 0.407, 0.214],
    "Ross49": ["05 44 56.81","+09 14 32.1",11.163, 1.130, 0.467, 0.162, 0.049],
    "Hilt_566": ["06 32 09.67","+03 34 44.3",10.787, 1.125, 0.673, 0.341, 0.211],
    "Ru-149F": ["07 24 14.02","-00 31 38.2", 13.119, 2.469, 0.867, 0.317, 0.166],
    "SA100-280": ["08 53 34.9","-00 37 38.5",11.689, 1.143, 0.308, 0.084, 0.003],
    "PG0918+029D": ["09 21 34.0","+02 46 39.0",11.937, 2.227, 0.817, 0.324, 0.166],
    "SA101-316": ["09 54 52.03","-00 18 34.5",11.438, 1.152, 0.309, 0.073, 0.007],
    "G162-66": ["10 33 42.81","-11 41 38.7", 13.227, -0.183, -0.387, -0.354, -0.303],
    "Feige_34": ["10 39 36.74","+43 06 09.2",11.423, -0.509, -0.508, -0.347, -0.265],
    "PG1047+003A": ["10 50 09.0","-00 01 08.0",13.303, 1.385, 0.519, 0.212, 0.087],
    "G163-50": ["11 07 59.97","-05 09 26.0", 13.266, 0.215, -0.277, -0.272, -0.271],
    "Feige66": ["12 37 23.52","+25 03 59.9",10.747, -0.345, -0.476, -0.367, -0.316],
    "SA104-428": ["12 41 41.29","-00 26 26.1",12.330, 2.153, 0.763, 0.279, 0.147],
    "PG1323-086D": ["13 26 05.26", "-08 50 35.7", 11.928, 1.210, 0.397, 0.132, 0.032],
    "Ross_838": ["14 01 44.48","+08 55 17.5",11.327, 1.277, 0.573, 0.239, 0.111],
    "PG1528+062B": ["15 30 39.55", "+06 01 13.1", 11.828, 1.235, 0.419, 0.143, 0.036],
    "G15-24": ["15 30 41.76","+08 23 40.4", 11.277, 1.035, 0.412, 0.151, 0.052],
    "BD+33 2642": ["15 51 59.89","+32 56 54.3",10.979, -0.018, -0.332, -0.284, -0.212],
    "Ross_530": ["16 19 51.65","+22 38 20.4",11.319, 1.273, 0.558, 0.229, 0.103],
    "Wolf_629": ["16 55 25.22","-08 19 21.3",11.129, 3.013, 1.413, 1.466, 0.648],
    "SA109-381": ["17 44 12.27","-00 20 32.8",11.514, 1.477, 0.547, 0.223, 0.094],
    "Ross_711": ["18 35 19.17","+28 41 55.4",11.295, 0.837, 0.282, 0.104, 0.015],
    "SA110-232": ["18 40 52.33", "+00 01 54.8", 12.287, 1.390, 0.552, 0.237, 0.094],
    "SA111-1925": ["19 37 28.64","+00 25 02.8",12.345, 1.397, 0.200, 0.061, 0.051],
    "Wolf_1346": ["20 34 21.88","+25 03 49.8",11.753, -0.016, -0.351, -0.309, -0.291],
    "SA112-805": ["20 42 46.75","+00 16 08.1",12.174, 1.183, -0.087, -0.135, -0.090],
    "BD+28 4211": ["21 51 11.02","+28 51 50.4",10.750, -0.517, -0.511, -0.379, -0.313],
    "G93-48": ["21 52 25.37","+02 23 19.6", 12.96, 10.107, -0.308, -0.307, -0.261],
    "SA114-656": ["22 41 35.07","+01 11 09.8",12.326, 1.961, 0.756, 0.293, 0.156],
    "GD_246": ["23 12 21.62","+10 47 04.3",13.346, -0.491, -0.504, -0.378, -0.367],
    "PG2336+004B": ["23 38 38.26","+00 42 46.4", 12.312, 1.101, 0.336, 0.100, 0.014]
}

In [ ]:
std_name = img_std.header["OBJECT"].split("STD_")[1]

In [ ]:
ra_std, dec_std = phot_standards[std_name][0:2]
coord_std = coordinates.SkyCoord(ra_std, dec_std, unit=(u.hour, u.degree))

In [ ]:
phot_standards[std_name]

In [ ]:
std_u, std_g, std_r, std_i, std_z = get_ugriz(phot_standards[std_name][2:])

In [ ]:
std_mag = {
    "u": std_u,
    "g": std_g,
    "r": std_r,
    "i": std_i,
    "z": std_z,
}[filt_letter]

# Image processing

In [ ]:
img_std.header["RAdeg"], img_std.header["DECdeg"]

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=img_std.wcs))
show_image(img_std, dpi=100, fig=fig, ax=ax)

In [ ]:
show_image(img_std, dpi=100)

In [ ]:
bkg_std = sep.Background(swap_byteorder(img_std.data))

In [ ]:
img_nobkg_std = swap_byteorder(img_std.data) - bkg_std

In [ ]:
show_image(bkg_std.back(), dpi=100)

In [ ]:
objects = sep.extract(img_nobkg_std, 5, err=bkg_std.globalrms)

In [ ]:
coords = img_std.wcs.pixel_to_world(objects["x"], objects["y"])

In [ ]:
idx_std = np.argmin(coords.separation(coord_std))

In [ ]:
coords[idx_std], coord_std

In [ ]:
objects[idx_std]

In [ ]:
coords

In [ ]:
r_std = 100

In [ ]:
plot_sources(objects, img_nobkg_std)
plt.scatter(objects[idx_std]["x"], objects[idx_std]["y"])

In [ ]:
obj_std = objects[idx_std]

In [ ]:
radii = np.logspace(0, 2.5, 100)

N = len(radii)
flux_std, fluxerr_std, flag_std = sep.sum_circle(img_nobkg_std, np.full(N, obj_std["x"]), 
               np.full(N, obj_std["y"]), radii)
flux_std /= exp_time_std / gain
fluxerr_std /= exp_time_std / gain

plt.figure()
plt.scatter(radii, flux_std)
plt.xscale("log")
plt.axvline(100)


In [ ]:
idx_0_std = np.argmin(np.abs(radii - r_std))
flux_0_std = flux_std[idx_0_std]
flux_0_err_std = fluxerr_std[idx_0_std]

In [ ]:
mag0_std, mag0_err_std = to_mag(flux_0_std, flux_0_err_std, 0., atm_ext)
zero_point = std_mag - mag0_std
zero_point_err = atm_ext_err + mag0_err_std

In [ ]:
zero_point, zero_point_err

In [ ]:
fluxes = []
fluxerrs = []
flags = []
radii = [1, 2, 5, 10, 20]
for r in radii:
    flux, fluxerr, flag = sep.sum_circle(img_nobkg_std, objects['x'], objects['y'],
                                         r / plate_scale, err=bkg_std.globalrms)

    fluxes.append(flux * gain / exp_time_std)
    fluxerrs.append(fluxerr * gain  / exp_time_std)
    flags.append(flag)

# Panstarrs xmatch

In [ ]:
from astropy.table import Table

In [ ]:
panstarrs = Table.read("../survey_data/stds1_panstarrs.fits")

In [ ]:
idx_xmatch, dist_xmatch, filt_xmatch = xmatch(coords.ra, coords.dec, panstarrs["ra"].reshape(-1), panstarrs["dec"].reshape(-1), 1*u.arcsec)

In [ ]:
filt_xmatch

In [ ]:
np.sum(filt_xmatch)

In [ ]:
plt.figure()
plt.scatter(coords.ra[filt_xmatch], coords.dec[filt_xmatch])
plt.scatter(panstarrs["ra"][idx_xmatch[filt_xmatch]], panstarrs["dec"][idx_xmatch[filt_xmatch]], s=5)


In [ ]:
?to_mag

In [ ]:
mag0, mag0_err = to_mag(fluxes[-1][filt_xmatch], fluxerrs[-1][filt_xmatch], 0, atm_ext)

In [ ]:
allmags_errs = [to_mag(flux, fluxerr, 0, atm_ext) for flux, fluxerr in zip(fluxes, fluxerrs)]
allmags = [x[0] for x in allmags_errs]
allerrs = [x[1] for x in allmags_errs]

In [ ]:
len(objects)

In [ ]:
np.full(len(objects), 0.5).size

In [ ]:
img_nobkg_std

In [ ]:
r_half, flags_r_half = sep.flux_radius(img_nobkg_std, objects["x"], objects["y"], np.full(len(objects), 100.), 0.5)

In [ ]:
r_kron, flags_r_kron = sep.kron_radius(img_nobkg_std, objects["x"], objects["y"], objects["a"], objects["b"], objects["theta"], 6)

In [ ]:
plt.figure()
plt.hist(r_kron)

In [ ]:

plt.figure()
plt.hist(np.sqrt(objects["x2"]) * plate_scale)

In [ ]:
fig, axs =  plt.subplots(3, 3, figsize=(10, 10))

for i in range(3):
    for j in range(3):
        if i < j:
            axs[i][j].remove()
            continue
        plt.sca(axs[i][j])
        plt.scatter(allmags[i+1], -allmags[j] + allmags[i+1], c=np.log10(r_half))

        plt.xlabel(f"I{i+1}")
        plt.ylabel(f"I{i+1} - I{j}")


plt.colorbar()

In [ ]:
plt.figure()
plt.errorbar(allmags[2], allmags[3] - allmags[2], yerr=allerrs[2], fmt=".")
plt.scatter(allmags[2], allmags[3] - allmags[2], c=r_half, zorder=3)

x = np.linspace(-16, -8, 1000)
y = -0.1 - np.exp((x+9) * 0.6)
plt.plot(x, y)
plt.plot(x, -y)

In [ ]:

plt.figure()
plt.hist((r_half * plate_scale), bins=np.linspace(0, 2, 20))
plt.xlim(0, 3)

In [ ]:
mag = mag0 + zero_point

In [ ]:
fig, axs = plt.subplots(2, 1)

plt.sca(axs[0])
plt.scatter(mag, panstarrs[filt_letter + "MeanPSFMag"][idx_xmatch[filt_xmatch]])
plt.sca(axs[1])

plt.errorbar(mag, panstarrs[filt_letter + "MeanPSFMag"][idx_xmatch[filt_xmatch]].reshape(-1) - mag, yerr=np.max(mag0_err, 0), fmt="o")
plt.axhline(0)

In [ ]:
plt.figure()
plt.scatter(mag0, mag0_err)

In [ ]:
zero_point

In [ ]:
atm_ext